In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import keras.datasets.mnist as Mnist
from PIL import Image
from keras.models import load_model

batch=60

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def get_image(path,max_count,step):
    filepath = path
    video = cv2.VideoCapture(filepath) #'' 사이에 사용할 비디오 파일의 경로 및 이름을 넣어주도록 함

    if not video.isOpened():
        print("Could not Open :", filepath)
        exit(0)
        
    length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = video.get(cv2.CAP_PROP_FPS)

    print("length :", length)
    print("width :", width)
    print("height :", height)
    print("fps :", fps)
 
 
    count = 0
    train_data=[]
    while(video.isOpened()):
        ret, image = video.read()
        if not ret: break
        if(int(video.get(1)) % (int(fps*step)) == 0):
            count+=1
            image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
            image=cv2.resize(image,(256,187))
            # image=cv2.flip(image,0)
            if image.max()>=2: result=image/225
            else: result=image
            train_data.append(np.array(result))
            # print(image.shape)
            # plt.imshow(image)
            # plt.show()
            # break
            if max_count==-1: continue
            if count==max_count+4: break
    video.release()
    return np.array(train_data[:-50])

def pre_work(images):
    step=30
    image=[]
    print(images.shape)
    l=images.shape[0]
    for i in range(l-(step-1)):
        image.append(images[i:i+step])
    return np.array(image)

def make_label(x,l):
    return np.zeros(l) if x==0 else np.ones(l)

def make_label_grad(x,l):
    label=np.zeros(l)
    end=25
    start=50
    for i in range(l-(start-1),l-(end-1)):
        label[i]=0.04*(i-(l-start))
    for i in range(l-(end-1),l):
        label[i]=1
    # for i in range(l-(start-1),l):
    #     label[i]=1
    return label

In [ ]:
import os
dir_path = "/content/drive/MyDrive/캡디 운전 동영상/반디컷"
file_list=[]
for (root, directories, files) in os.walk(dir_path):
    for file in files:
        file_path = os.path.join(root, file)
        file_list.append(file_path)
        print(file_path)

def make_data(titles,count=-1,step=0.1):
    for title in titles:
        train_image=get_image(f'{title}',count,step)
        if len(train_image)<=55: return -1
        train_image=pre_work(train_image)
        train_label=make_label_grad(1,train_image.shape[0])
        print(train_image.shape)
        print(train_label)
        train_images.append(train_image)
        train_labels.append(train_label)
    return 1
# make_data(['sleep-night-giung','sleep-night-jun','sleep-night-moon','sleep-night-sung','sleep-sun-dad1','sleep-sun-dad2','wake-night-giung','wake-night-jun','wake-night-moon','wake-night-sung'],70)
# target=[]
# for i in range(1,26):
#     target.append(str(i))

/content/drive/MyDrive/캡디 운전 동영상/반디컷/8-1.mp4
/content/drive/MyDrive/캡디 운전 동영상/반디컷/8-2.mp4
/content/drive/MyDrive/캡디 운전 동영상/반디컷/8-3.mp4
/content/drive/MyDrive/캡디 운전 동영상/반디컷/8-5.mp4
/content/drive/MyDrive/캡디 운전 동영상/반디컷/8-7.mp4
/content/drive/MyDrive/캡디 운전 동영상/반디컷/8-8.mp4
/content/drive/MyDrive/캡디 운전 동영상/반디컷/8-11.mp4
/content/drive/MyDrive/캡디 운전 동영상/반디컷/8-12.mp4
/content/drive/MyDrive/캡디 운전 동영상/반디컷/8-13.mp4
/content/drive/MyDrive/캡디 운전 동영상/반디컷/8-14.mp4
/content/drive/MyDrive/캡디 운전 동영상/반디컷/8-15.mp4
/content/drive/MyDrive/캡디 운전 동영상/반디컷/8-16.mp4
/content/drive/MyDrive/캡디 운전 동영상/반디컷/8-17.mp4
/content/drive/MyDrive/캡디 운전 동영상/반디컷/8-18.mp4
/content/drive/MyDrive/캡디 운전 동영상/반디컷/8-19.mp4
/content/drive/MyDrive/캡디 운전 동영상/반디컷/11-1.mp4
/content/drive/MyDrive/캡디 운전 동영상/반디컷/11-2.mp4
/content/drive/MyDrive/캡디 운전 동영상/반디컷/11-3.mp4
/content/drive/MyDrive/캡디 운전 동영상/반디컷/11-4.mp4
/content/drive/MyDrive/캡디 운전 동영상/반디컷/11-5.mp4
/content/drive/MyDrive/캡디 운전 동영상/반디컷/11-6.mp4
/content/drive/MyDrive/캡디 운전 동영상/반디컷/11-

In [ ]:
from keras.layers import Dense, Input, Conv2D, Conv2DTranspose, Flatten, Reshape, MaxPool2D, GRU, GlobalAveragePooling2D,concatenate, LSTM, Flatten
from keras.models import Model
from keras.layers import BatchNormalization, Dropout, Activation, TimeDistributed

def vgg_conv2d_block(x,channel):
    x = TimeDistributed(Conv2D(channel, 3, padding="same"))(x)
    x = TimeDistributed(BatchNormalization())(x)
    x = TimeDistributed(Activation("relu"))(x)
    return x

def vgg_lstm_model():
    input=Input(shape=(30,187,256,3))
    c1 = vgg_conv2d_block(input, 16)
    c1 = vgg_conv2d_block(c1, 16)
    c1 = TimeDistributed(MaxPool2D(2))(c1)
    c1 = TimeDistributed(Dropout(0.1))(c1)

    c2 = vgg_conv2d_block(c1, 32)
    c2 = vgg_conv2d_block(c2, 32)
    c2 = TimeDistributed(MaxPool2D(2))(c2)
    c2 = TimeDistributed(Dropout(0.1))(c2)

    c3 = vgg_conv2d_block(c2, 64)
    c3 = vgg_conv2d_block(c3, 64)
    c3 = vgg_conv2d_block(c3, 64)
    c3 = TimeDistributed(MaxPool2D(2))(c3)
    c3 = TimeDistributed(Dropout(0.1))(c3)

    c4 = vgg_conv2d_block(c3, 128)
    c4 = vgg_conv2d_block(c4, 128)
    c4 = vgg_conv2d_block(c4, 128)
    c4 = TimeDistributed(MaxPool2D(2))(c4)
    c4 = TimeDistributed(Dropout(0.1))(c4)

    c5 = vgg_conv2d_block(c4, 256)
    c5 = vgg_conv2d_block(c5, 256)
    c5 = vgg_conv2d_block(c5, 256)
    c5 = TimeDistributed(MaxPool2D(2))(c5)
    c5 = TimeDistributed(Dropout(0.1))(c5)

    embed=TimeDistributed(GlobalAveragePooling2D())(c5)
    lstm=LSTM(256)(embed)
    # dense=Dense(10,activation='relu')(lstm)
    output=Dense(1)(lstm)

    return Model(input,output)

def conv2d_block(x, channel):
    x = Conv2D(channel, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

def image_embedding():
    inputs=Input(shape=(187,256,3))
    c1 = conv2d_block(inputs, 16)
    c1 = conv2d_block(c1, 16)
    p1 = MaxPool2D(2)(c1)
    p1 = Dropout(0.1)(p1)

    c2 = conv2d_block(p1, 32)
    c2 = conv2d_block(c2, 32)
    p2 = MaxPool2D(2)(c2)
    p2 = Dropout(0.1)(p2)

    c3 = conv2d_block(p2, 64)
    c3 = conv2d_block(c3, 64)
    c3 = conv2d_block(c3, 64)
    p3 = MaxPool2D(2)(c3)
    p3 = Dropout(0.1)(p3)

    c4 = conv2d_block(p3, 128)
    c4 = conv2d_block(c4, 128)
    c4 = conv2d_block(c4, 128)
    p4 = MaxPool2D(2)(c4)
    p4 = Dropout(0.1)(p4)

    c5 = conv2d_block(p4,256)

    u6 = Conv2DTranspose(128, 2, 2, padding="valid", output_padding=(1,0))(c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(0.1)(u6)
    c6 = conv2d_block(u6, 128)

    u7 = Conv2DTranspose(64, 2, 2)(c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(0.1)(u7)
    c7 = conv2d_block(u7, 64)

    u8 = Conv2DTranspose(32, 2, 2, padding="valid", output_padding=(1,0))(c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(0.1)(u8)
    c8 = conv2d_block(u8, 32)

    u9 = Conv2DTranspose(16, 2, 2, padding="valid", output_padding=(1,0))(c8)
    u9 = concatenate([u9, c1])
    u9 = Dropout(0.1)(u9)
    c9 = conv2d_block(u9, 16)

    outputs = Conv2D(3, 1, activation="sigmoid")(c9)
    return Model(inputs,outputs)

def image_encoder():
    inputs=Input(shape=(187,256,3))
    c1 = conv2d_block(inputs, 16)
    c1 = conv2d_block(c1, 16)
    p1 = MaxPool2D(2)(c1)
    p1 = Dropout(0.1)(p1)

    c2 = conv2d_block(p1, 32)
    c2 = conv2d_block(c2, 32)
    p2 = MaxPool2D(2)(c2)
    p2 = Dropout(0.1)(p2)

    c3 = conv2d_block(p2, 64)
    c3 = conv2d_block(c3, 64)
    c3 = conv2d_block(c3, 64)
    p3 = MaxPool2D(2)(c3)
    p3 = Dropout(0.1)(p3)

    c4 = conv2d_block(p3, 128)
    c4 = conv2d_block(c4, 128)
    c4 = conv2d_block(c4, 128)
    p4 = MaxPool2D(2)(c4)
    p4 = Dropout(0.1)(p4)

    c5 = conv2d_block(p4,256)
    output=GlobalAveragePooling2D()(c5)
    return Model(inputs,output)

def image_embedding_without_unet():
    inputs=Input(shape=(187,256,3))
    c1 = conv2d_block(inputs, 16)
    c1 = conv2d_block(c1, 16)
    p1 = MaxPool2D(2)(c1)
    p1 = Dropout(0.1)(p1)

    c2 = conv2d_block(p1, 32)
    c2 = conv2d_block(c2, 32)
    p2 = MaxPool2D(2)(c2)
    p2 = Dropout(0.1)(p2)

    c3 = conv2d_block(p2, 64)
    c3 = conv2d_block(c3, 64)
    c3 = conv2d_block(c3, 64)
    p3 = MaxPool2D(2)(c3)
    p3 = Dropout(0.1)(p3)

    c4 = conv2d_block(p3, 128)
    c4 = conv2d_block(c4, 128)
    c4 = conv2d_block(c4, 128)
    p4 = MaxPool2D(2)(c4)
    p4 = Dropout(0.1)(p4)

    c11 = conv2d_block(p4, 256)
    c11 = conv2d_block(c11, 256)
    c11 = conv2d_block(c11, 256)
    p11 = MaxPool2D(2)(c11)
    p11 = Dropout(0.1)(p11)

    c5 = conv2d_block(p11,512)

    t1=GlobalAveragePooling2D()(c5)
    t1=Reshape((1,1,512))(t1)

    u10 = Conv2DTranspose(512, 2, 2, padding="valid", output_padding=(0,0))(t1)
    u10 = Dropout(0.1)(u10)
    c10 = conv2d_block(u10, 512)
    c10=Conv2D(512,3,padding='same',strides=(2,1))(c10)

    u12 = Conv2DTranspose(512, 2, 2, padding="valid", output_padding=(0,0))(c10)
    u12 = Dropout(0.1)(u12)
    c12 = conv2d_block(u12, 512)

    u13 = Conv2DTranspose(256, 2, 2, padding="valid", output_padding=(1,0))(c12)
    u13 = Dropout(0.1)(u13)
    c13 = conv2d_block(u13, 256)

    u14 = Conv2DTranspose(128, 2, 2, padding="valid", output_padding=(1,0))(c13)
    u14 = Dropout(0.1)(u14)
    c14 = conv2d_block(u14, 128)

    u7 = Conv2DTranspose(64, 2, 2, padding="valid", output_padding=(1,0))(c14)
    u7 = Dropout(0.1)(u7)
    c7 = conv2d_block(u7, 64)

    u8 = Conv2DTranspose(32, 2, 2, padding="valid", output_padding=(0,0))(c7)
    u8 = Dropout(0.1)(u8)
    c8 = conv2d_block(u8, 32)

    u9 = Conv2DTranspose(16, 2, 2, padding="valid", output_padding=(1,0))(c8)
    u9 = Dropout(0.1)(u9)
    c9 = conv2d_block(u9, 16)

    u15 = Conv2DTranspose(16, 2, 2, padding="valid", output_padding=(1,0))(c9)
    u15 = Dropout(0.1)(u15)
    c15= conv2d_block(u15, 16)

    outputs = Conv2D(3, 1, activation="sigmoid")(c15)
    return Model(inputs,outputs)

model=vgg_lstm_model()
model.summary()


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 30, 187, 256, 3)  0         
                             ]                                   
                                                                 
 time_distributed_100 (TimeD  (None, 30, 187, 256, 16)  448      
 istributed)                                                     
                                                                 
 time_distributed_101 (TimeD  (None, 30, 187, 256, 16)  64       
 istributed)                                                     
                                                                 
 time_distributed_102 (TimeD  (None, 30, 187, 256, 16)  0        
 istributed)                                                     
                                                                 
 time_distributed_103 (TimeD  (None, 30, 187, 256, 16)  232

In [ ]:
model=load_model('./drive/MyDrive/capdi/acci_pred2-14.h5')
# model.load_weights(checkpoint_path)

In [ ]:
class mse(tf.keras.losses.Loss):

  def call(self, y_true, y_pred):
    err = y_true - y_pred
    loss = tf.math.reduce_mean(tf.math.square(err))

    return loss
loss=mse()

In [ ]:
model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(), metrics=['mse'])

In [ ]:
tf.config.experimental_run_functions_eagerly(True)

In [ ]:
print(train_images.shape)
print(train_labels.shape)

In [ ]:
import random
train_images=np.array(train_images)
train_labels=np.array(train_labels)
print(train_images.shape)
print(train_labels.shape)


train_images=np.concatenate(train_images,axis=0)
train_labels=np.concatenate(train_labels,axis=0)



In [ ]:
test_images=np.concatenate([train_images[48:96],train_images[144:192]],axis=0)
test_labels=np.concatenate([train_labels[48:96],train_labels[144:192]],axis=0)
train_images=np.concatenate([train_images[0:48],train_images[96:144]],axis=0)
train_labels=np.concatenate([train_labels[0:48],train_labels[96:144]],axis=0)

In [ ]:
train_images.shape
plt.imshow(train_images[50,0,...])
plt.show()

In [ ]:
shuffle_list=list(zip(train_images,train_labels))
random.shuffle(shuffle_list)
shuffle_list=np.array(shuffle_list)

train_images=shuffle_list[:,0]
train_labels=shuffle_list[:,1]

train_images=np.array(train_images.tolist())
train_labels=np.array(train_labels.tolist())
print(train_images.shape)
print(train_labels.shape)

In [ ]:
print(train_images.shape)
print(train_labels.shape)

(648, 5, 187, 256, 3)
(648,)


In [ ]:
print(train_images[0].shape)
print(len(train_images))

In [ ]:
# hist=model.fit(train_images,train_labels,batch_size=10,epochs=25,verbose=1)
hist=model.fit(train_images,train_labels,batch_size=10,epochs=25,verbose=1)

InternalError: ignored

In [ ]:
print(train_images.shape)
pred=[]
for image in train_images:
    # print(np.array([image]).shape)
    plt.imshow(image[-1])
    plt.show()
    print(model.predict(np.array([image])))

In [ ]:
# plt.imshow(pred[0])
# plt.show()
# print(pred.shape)
# plt.plot(pred.reshape(-1))
# plt.show()
# plt.plot(train_labels)
# plt.show()
# plt.imshow(train_images[1,0,...])
# plt.show()
print(pred)
# plt.plot(pred)

[array([[0.59178656]], dtype=float32), array([[0.59178656]], dtype=float32), array([[0.59178656]], dtype=float32), array([[0.59178656]], dtype=float32), array([[0.59178656]], dtype=float32), array([[0.59178656]], dtype=float32), array([[0.59178656]], dtype=float32), array([[0.59178656]], dtype=float32), array([[0.59178656]], dtype=float32), array([[0.59178656]], dtype=float32), array([[0.59178656]], dtype=float32), array([[0.59178656]], dtype=float32), array([[0.59178656]], dtype=float32), array([[0.59178656]], dtype=float32), array([[0.59178656]], dtype=float32), array([[0.59178656]], dtype=float32), array([[0.59178656]], dtype=float32), array([[0.59178656]], dtype=float32), array([[0.59178656]], dtype=float32), array([[0.59178656]], dtype=float32), array([[0.59178656]], dtype=float32), array([[0.59178656]], dtype=float32), array([[0.59178656]], dtype=float32), array([[0.59178656]], dtype=float32), array([[0.59178656]], dtype=float32), array([[0.59178656]], dtype=float32), array([[0.5

In [ ]:
model.evaluate(train_images,train_labels)

In [ ]:
model.save('./drive/MyDrive/capdi/acci_pred2.h5')

In [ ]:
model=load_model('./drive/MyDrive/capdi/auto.h5')

In [ ]:
fine_model=load_model('./drive/MyDrive/capdi/fine-vgg-lstm.h5')

In [ ]:
# print(last_layer.output.shape)
weight=model.get_weights()

In [ ]:
fine_model.summary()

In [ ]:
pre_weight=fine_model.get_weights()
pre_weight[:66]=weight[:66]
fine_model.set_weights(pre_weight)

In [ ]:
fine_model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
# fine_model.save('fine-vgg-lstm2.h5')

NameError: ignored

In [ ]:
hist=fine_model.fit(train_images,train_labels,batch_size=10,epochs=25,verbose=1)

In [ ]:
model.save('./drive/MyDrive/capdi/acci_pred2.h5')

In [ ]:
pred=fine_model.predict(train_images)

In [ ]:
plt.plot(pred.reshape(-1))
plt.show()

In [ ]:
pred=fine_model.evaluate(train_images,train_labels)

In [ ]:
for i,target in enumerate(file_list[2:3]):
    # train_images=[]
    # train_labels=[]
    # data_set=[]
    # ok=make_data([target],600,0.1)
    # if ok==-1:
    #     print("errrrrr "+target)
    #     continue
    # train_images=np.array(train_images)
    # train_labels=np.array(train_labels)
    # train_images=np.concatenate(train_images,axis=0)
    # train_labels=np.concatenate(train_labels,axis=0)
    # checkpoint_path = "training_8/cp-"+str(i)+"-{epoch:04d}.ckpt"
    # checkpoint_dir = os.path.dirname(checkpoint_path)#previoulsy, checkpoint_path = "training_1/cp.ckpt"
    # cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
    #                                 save_weights_only=True,
    #                                 verbose=1,period=5)# Train the model with the new callback
    hist=model.fit(train_images[...],train_labels[...],batch_size=2,epochs=10,verbose=1)
    # model.save('./drive/MyDrive/capdi/acci_pred4-'+-str(i)+'.h5')

Epoch 1/10
31/31 [==============================] - 10s 327ms/step - loss: 0.0114 - mse: 0.0114
Epoch 2/10
31/31 [==============================] - 10s 317ms/step - loss: 0.0112 - mse: 0.0112
Epoch 3/10
31/31 [==============================] - 9s 306ms/step - loss: 0.0083 - mse: 0.0083
Epoch 4/10
31/31 [==============================] - 9s 306ms/step - loss: 0.0090 - mse: 0.0090
Epoch 5/10
31/31 [==============================] - 9s 306ms/step - loss: 0.0079 - mse: 0.0079
Epoch 6/10
31/31 [==============================] - 9s 302ms/step - loss: 0.0079 - mse: 0.0079
Epoch 7/10
31/31 [==============================] - 9s 302ms/step - loss: 0.0053 - mse: 0.0053
Epoch 8/10
31/31 [==============================] - 9s 304ms/step - loss: 0.0047 - mse: 0.0047
Epoch 9/10
31/31 [==============================] - 9s 305ms/step - loss: 0.0056 - mse: 0.0056
Epoch 10/10
31/31 [==============================] - 9s 305ms/step - loss: 0.0044 - mse: 0.0044


In [ ]:
model.save('./drive/MyDrive/capdi/acci_pred2.h5')

In [ ]:
for target in file_list[2:3]:
    # train_images=[]
    # train_labels=[]
    # data_set=[]
    # ok=make_data([target],600,0.1)
    # if ok==-1:
    #     print("errrrrr "+target)
    #     continue
    # train_images=np.array(train_images)
    # train_labels=np.array(train_labels)
    # train_images=np.concatenate(train_images,axis=0)
    # train_labels=np.concatenate(train_labels,axis=0)
    # print(train_images.shape)
    # pred=[]
    for i, image in enumerate(train_images):
        # print(np.array([image]).shape)
        # plt.imshow(image[-1])
        # plt.show()
        # break
        print(train_labels[i],model.predict(np.array([image])))

1/1 [==============================] - 0s 103ms/step
0.0 [[0.09817733]]
1/1 [==============================] - 0s 98ms/step
0.0 [[0.09266242]]
1/1 [==============================] - 0s 111ms/step
0.0 [[0.07874554]]
1/1 [==============================] - 0s 103ms/step
0.0 [[0.07108986]]
1/1 [==============================] - 0s 102ms/step
0.0 [[0.06724118]]
1/1 [==============================] - 0s 95ms/step
0.0 [[0.07629471]]
1/1 [==============================] - 0s 99ms/step
0.0 [[0.08710367]]
1/1 [==============================] - 0s 102ms/step
0.0 [[0.09047518]]
1/1 [==============================] - 0s 94ms/step
0.0 [[0.09871182]]
1/1 [==============================] - 0s 95ms/step
0.0 [[0.10559178]]
1/1 [==============================] - 0s 95ms/step
0.0 [[0.10061308]]
1/1 [==============================] - 0s 95ms/step
0.0 [[0.10912535]]
1/1 [==============================] - 0s 97ms/step
0.04 [[0.1252557]]
1/1 [==============================] - 0s 97ms/step
0.08 [[0.14728141]]


In [ ]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!wandb login

wandb: Currently logged in as: semteo04. Use `wandb login --relogin` to force relogin


In [ ]:
!wandb login

wandb: Currently logged in as: semteo04. Use `wandb login --relogin` to force relogin


In [ ]:
import wandb
from wandb.keras import WandbCallback
run = None
run = wandb.init(project='danger')

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: semteo04. Use `wandb login --relogin` to force relogin
